In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader

In [63]:
# Check if CUDA is available
if torch.cuda.is_available():
    # Set the default tensor type to CUDA tensors
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

    # Define the device as the first visible cuda device if available
    device = torch.device("cuda:0")
    print(f"Running on {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("Running on CPU")


Running on NVIDIA GeForce RTX 3080


In [64]:
# Data loaders for your deep fake and genuine images dataset
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # Normalization with ImageNet mean and std
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [65]:
# Updated paths for the datasets
train_dataset_path = "C:/Users/Henry/Downloads/archive/Dataset/train"
val_dataset_path = "C:/Users/Henry/Downloads/archive/Dataset/validation"
# test_dataset_path = "C:/Users/Henry/Downloads/archive/Dataset/test"  # Uncomment this if you have a test dataset

# Ensure train_transform is defined correctly
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # Add other transformations as needed
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dataset_path, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, generator=torch.Generator(device='cuda'))

val_dataset = datasets.ImageFolder(val_dataset_path, transform=train_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)



In [66]:
# Initialize ResNet model and modify the last layer
model = resnet50(pretrained=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)  # Assuming 2 classes: genuine and deep fake
model = model.to(device)

c:\Users\Henry\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Henry\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [67]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [68]:
for epoch in range(1):  # 10 epochs, adjust as needed
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)


        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/10], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")


Epoch [1/10], Step [100/4376], Loss: 0.3308
Epoch [1/10], Step [200/4376], Loss: 0.1084
Epoch [1/10], Step [300/4376], Loss: 0.1632
Epoch [1/10], Step [400/4376], Loss: 0.1661
Epoch [1/10], Step [500/4376], Loss: 0.0956
Epoch [1/10], Step [600/4376], Loss: 0.0843
Epoch [1/10], Step [700/4376], Loss: 0.1635
Epoch [1/10], Step [800/4376], Loss: 0.1375
Epoch [1/10], Step [900/4376], Loss: 0.1159
Epoch [1/10], Step [1000/4376], Loss: 0.1403
Epoch [1/10], Step [1100/4376], Loss: 0.1325
Epoch [1/10], Step [1200/4376], Loss: 0.2016
Epoch [1/10], Step [1300/4376], Loss: 0.1030
Epoch [1/10], Step [1400/4376], Loss: 0.0526
Epoch [1/10], Step [1500/4376], Loss: 0.0818
Epoch [1/10], Step [1600/4376], Loss: 0.1927
Epoch [1/10], Step [1700/4376], Loss: 0.1011
Epoch [1/10], Step [1800/4376], Loss: 0.0543
Epoch [1/10], Step [1900/4376], Loss: 0.0775
Epoch [1/10], Step [2000/4376], Loss: 0.0760
Epoch [1/10], Step [2100/4376], Loss: 0.1158
Epoch [1/10], Step [2200/4376], Loss: 0.1335
Epoch [1/10], Step 

In [69]:
# Validation Loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    print(f"Validation Accuracy: {100 * correct / total}%")


Validation Accuracy: 94.68651719590139%
